In [1]:
import pandas as pd
import geopandas as gpd
import folium
from folium import plugins
from folium.plugins import MeasureControl
import fiona
import json
from keplergl import KeplerGl
from geopy.distance import geodesic


# Show Prediction Results

In [5]:

# minimum distance between points to be plotted on the map
min_distance = 500

# Enable KML support which is disabled by default
fiona.drvsupport.supported_drivers['kml'] = 'rw' # enable KML support which is disabled by default
fiona.drvsupport.supported_drivers['KML'] = 'rw' # enable KML support which is disabled by default
fiona.drvsupport.supported_drivers['LIBKML'] = 'rw' # enable KML support which is disabled by default

# Load KML files using geopandas
path = './Visualization/'
kml_files = ['full_source.kml', 'source.kml', 'true.kml', 'pred_LSTM.kml', 'pred_seq_attention.kml']  # Add paths to KML files
gdf_list = [gpd.read_file(path + kml) for kml in kml_files]

# Convert GeoDataFrames to GeoJSON strings
gdf_json_list = [gdf.to_crs(epsg='4326').to_json() for gdf in gdf_list]

# Parse the GeoJSON strings
gdf_geojson_list = [json.loads(gdf_json) for gdf_json in gdf_json_list]

# crea a list of colors to use for the layers
colors = ['grey', 'blue', 'red', 'green', 'orange']

# creat map frame
f = folium.Figure(width=1000, height=500)

# Create a folium map with center location and zoom level on the map
map_obj = folium.Map(location=[gdf_geojson_list[0]['features'][0]['geometry']['coordinates'][0][1],
                               gdf_geojson_list[0]['features'][0]['geometry']['coordinates'][0][0]],
                     zoom_start=8).add_to(f)

# add measure control to the map
map_obj.add_child(MeasureControl())

# Define basemap layers with attribution
folium.TileLayer('OpenStreetMap').add_to(map_obj)
folium.TileLayer('CartoDB Positron').add_to(map_obj)

# # Create FeatureGroups for each layer
# line_layer = folium.FeatureGroup(name='Lines')
# point_layer = folium.FeatureGroup(name='Points')

# # Add FeatureGroups to the map
# line_layer.add_to(map_obj)
# point_layer.add_to(map_obj)


# check if the geometry is line or point
grouplayers_lines = {}
grouplayers_points = {}
for i in range(len(gdf_list)):
    # add layer to line layer group
    name = kml_files[i].split('.')[0]
    grouplayers_lines[kml_files[i].split('.')[0]] = folium.FeatureGroup(name = name).add_to(map_obj)
    # grouplayers_lines[kml_files[i].split('.')[0]] = plugins.FeatureGroupSubGroup(line_layer, kml_files[i].split('.')[0])
    # map_obj.add_child(grouplayers_lines[kml_files[i].split('.')[0]])
    # Add GeoJSON layers to the map
    # for i, gdf_geojson in enumerate(gdf_geojson_list):
    gdf_geojson = gdf_geojson_list[i]
    line_path = folium.GeoJson(
        gdf_geojson,
        name = name,
        style_function=lambda x, color=colors[i]: {
            'color': color,
            'weight': 3,
            'opacity': 1,
            }
    ).add_to(grouplayers_lines[kml_files[i].split('.')[0]])
    # check if name includes pred
    if 'pred' in name:
        # add arrow to the line
        attr = {"fill": colors[i], "font-weight": "bold", "font-size": "24"}
        folium.plugins.PolyLineTextPath(
            line_path, "\u25BA", repeat=True, offset=7, attributes=attr
        ).add_to(grouplayers_lines[kml_files[i].split('.')[0]])
        # folium.plugins.AntPath(
        # locations=line_path, reverse="True", dash_array=[20, 30]
        # ).add_to(grouplayers_lines[kml_files[i].split('.')[0]])
        
        


# fit the map to the bounds of the lines
map_obj.fit_bounds(map_obj.get_bounds())

# Add Layer Control
folium.LayerControl(collapsed=True).add_to(map_obj)

# Display the map
map_obj


# Show all trajectories on map

In [ ]:

df = pd.read_csv('DataSet/DataSet/test_fix.csv', header=None, names=['time', 'id', 'sog', 'lon', 'lat', 'cog', 'delta_time', 'delta_lat', 'delta_long'])

# Convert 'time' column to datetime format
df['time'] = pd.to_datetime(df['time'])

# convert time to string
df['time'] = df['time'].astype(str)


# creat a kepler.gl map in a frame
map_1 = KeplerGl(height=400, data={"data_1": df})
# make the center of the map to be the center of the data
center = [df['lat'].mean(), df['lon'].mean()]
map_1.config = {'version': 'v1', 'config': {'mapState': {'latitude': center[0], 'longitude': center[1], 'zoom': 10}}}


# show the map
# map_1.save_to_html(file_name="kepler.html")